In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim

import torchvision 
from torch.utils.data import DataLoader,Dataset
import torch.utils.data as utils
from torchvision import transforms 

import pandas as pd

In [2]:
import matplotlib.image as mpimg


In [3]:
data_dir = "aerial-cactus-identification/"
train_dir = data_dir + "train/"
test_dir = data_dir + "test"

In [4]:
labels = pd.read_csv("aerial-cactus-identification/train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [5]:
labels

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
...,...,...
17495,ffede47a74e47a5930f81c0b6896479e.jpg,0
17496,ffef6382a50d23251d4bc05519c91037.jpg,1
17497,fff059ecc91b30be5745e8b81111dc7b.jpg,1
17498,fff43acb3b7a23edcc4ae937be2b7522.jpg,0


In [6]:
import os 

class ImageData(Dataset):
    def __init__(self,df,data_dir,transform):
        super().__init__()
        
        self.df = df
        self.transform = transform
        self.data_dir = data_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        print("yo",img_name,index)
        img_path = os.path.join(self.data_dir,img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image,label
        
    

In [7]:
data_trans = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.ToTensor()
])

train_data = ImageData(df = labels,data_dir=train_dir,transform=data_trans)
trainloader = DataLoader(dataset = train_data,batch_size = 2)
train_set, val_set = torch.utils.data.random_split(trainloader.dataset, [16500,1000])


In [8]:
!pip install efficientnet_pytorch


In [9]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')


In [10]:
for param in model.parameters():
    param.requires_grad = True

In [11]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs,1)

optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

In [12]:
loss_log = []

for epoch in range(5):
    model.train()
    for ii ,(data,target) in enumerate(trainloader):
        target = target.float()
        
        optimizer.zero_grad()
        output = model(data)
        
        m = nn.Sigmoid()
        loss = loss_func(m(output),target)
        loss.backward()
        
        optimizer.step()
        
        if ii%1000 == 0:
            loss_log.append(loss.item())
            
            
        print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

yo 0004be2cfeaba1c0361d39e2b000257b.jpg 0
yo 000c8a36845c0208e833c79c1bffedd1.jpg 1


/home/kunal/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch: 1 - Loss: 0.724496
yo 000d1e9a533f62e55c289303b072733d.jpg 2
yo 0011485b40695e9138e92d0b3fb55128.jpg 3
Epoch: 1 - Loss: 0.705073
yo 0014d7a11e90b62848904c1418fc8cf2.jpg 4
yo 0017c3c18ddd57a2ea6f9848c79d83d2.jpg 5
Epoch: 1 - Loss: 0.508536
yo 002134abf28af54575c18741b89dd2a4.jpg 6
yo 0024320f43bdd490562246435af4f90b.jpg 7
Epoch: 1 - Loss: 0.983998
yo 002930423b9840e67e5a54afd4768a1e.jpg 8
yo 00351838ebf6dff6e53056e00a1e307c.jpg 9


KeyboardInterrupt: 

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [ ]:
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    output = model(data)    

    pred = torch.sigmoid(output)
    predicted_vals = pred > 0.5
    predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)